# ETL Pipeline Preparation

In this notebook we will be working on building ETL Pipeline for two csv files. Dataset is provided by Figure Eight Company
### 1. Import necessary python libraries and load datasets.
- Import Python libraries
- Load `disaster_messages.csv` into a dataframe.
- Load `disaster_categories.csv` into a dataframe and inspect the first few lines.

In [1]:
#Import Python Modules

#To Handle Datasets
import pandas as pd
import numpy as np

#Sqlite Database
import sqlite3
from sqlalchemy import create_engine

In [2]:
#Read disaster_messages dataset
df_msg = pd.read_csv('../data/disaster_messages.csv')
df_msg.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
#print info of daaset
df_msg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        26248 non-null  int64 
 1   message   26248 non-null  object
 2   original  10184 non-null  object
 3   genre     26248 non-null  object
dtypes: int64(1), object(3)
memory usage: 820.4+ KB


- observation: Except Original column all other columns doesn't have any null values

In [4]:
#Count number of nulls in original column
df_msg['original'].isnull().sum()

16064

In [5]:
#Display number of Unique values for each column in the dataset
for col in df_msg.columns:
    print("Number of Unique {} in the dataset are : {}".format(col,df_msg[col].nunique()))

Number of Unique id in the dataset are : 26180
Number of Unique message in the dataset are : 26177
Number of Unique original in the dataset are : 9630
Number of Unique genre in the dataset are : 3


In [6]:
#Read disaster_categories dataset
df_cat = pd.read_csv('../data/disaster_categories.csv')
df_cat.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [7]:
#print info of daaset
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          26248 non-null  int64 
 1   categories  26248 non-null  object
dtypes: int64(1), object(1)
memory usage: 410.2+ KB


- Observation - No Null Values in this dataset

In [8]:
df_cat['categories'][0]

'related-1;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-0;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-0;floods-0;storm-0;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0'

- Observation: categories column can be split into multiple columns as its hard to figure out the category data like this

## 2.Split the category column

In [9]:
#split the categories with `;` into multiple columns. 
category = df_cat['categories'].str.split(';',expand=True)

In [10]:
category

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26243,related-0,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
26244,related-0,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
26245,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
26246,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-1,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


Need to get correct column headers instead of Numbers

In [11]:
#Extract the first row of the dataset to get column Headers
col_names = category.iloc[ 0, :]
print(col_names.head())

#Empty array
col_headers = []

#Iterate through this for loop to strip the number from the Header. Instead of related-1 we need header as related
for name in col_names:
    name1 = name.rstrip('- 0 1')
    col_headers.append(name1)
    
print(col_headers)

0         related-1
1         request-0
2           offer-0
3     aid_related-0
4    medical_help-0
Name: 0, dtype: object
['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [12]:
category.columns = col_headers
category.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


For Row values we just need to display the numbers
 - Instead of `related-1` we need to display `1'

In [13]:
#Convert each column values into numeric instead of string

for col in category.columns:
    # Keep the Last value alone for each row
    category[col]= category[col].str[-1]
    #convert the column into numeric instead of string
    category[col] = pd.to_numeric(category[col].str[-1],errors='coerce')
    
category.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


concat this with df_cat dataset and drop the categories column 

In [14]:
#Drop categories column for the df_cat dataset
df_cat = df_cat.drop(['categories'],axis=1)
df_cat.head()

,id
0,2
1,7
2,8
3,9
4,12


In [15]:
df_category = pd.concat([df_cat,category],axis=1)
df_category.head()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#Display number of Unique values for each column in the dataset
for col in df_cat.columns:
    print("Number of Unique {} in the dataset are : {}".format(col,df_cat[col].nunique()))

Number of Unique id in the dataset are : 26180


- Observation: Both Dataset have 26180 unique id's and we can combine the datset based on id column

## 3.Merge Both Datsets with help of id column

In [17]:
df_all = pd.merge(df_msg, df_category, on="id")
df_all.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 4. Describe Dataset

In [18]:
df_all.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.00000,26386.000000,26386.000000,26386.00000,26386.000000,...,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000
mean,15217.885886,0.775032,0.171038,0.004586,0.415144,0.07955,0.049989,0.027477,0.01785,0.032707,...,0.011711,0.043773,0.278292,0.082506,0.093383,0.010687,0.093269,0.020200,0.052263,0.193777
std,8823.741128,0.435692,0.376549,0.067564,0.492756,0.27060,0.217926,0.163471,0.13241,0.177871,...,0.107583,0.204594,0.448166,0.275139,0.290974,0.102828,0.290815,0.140687,0.222560,0.395264
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7438.250000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15650.500000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22916.750000,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.000000,2.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


- Observation : we can see Related column has value as 2 but we should either have 0 or 1 for each category column

In [19]:
#Finding the values from the dataset which have related=2
df_all[df_all['related']==2]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
117,146,Dans la zone de Saint Etienne la route de Jacm...,Nan zon st. etine rout jakmel la bloke se mize...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
221,263,. .. i with limited means. Certain patients co...,t avec des moyens limites. Certains patients v...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307,373,The internet caf Net@le that's by the Dal road...,Cyber cafe net@le ki chita rout de dal tou pr ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
462,565,"Bonsoir, on est a bon repos aprs la compagnie ...",Bonswa nou nan bon repo apri teleko nan wout t...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
578,700,URGENT CRECHE ORPHANAGE KAY TOUT TIMOUN CROIX ...,r et Salon Furterer. mwen se yon Cosmtologue. ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20465,23662,transferred to a sanitary landfill site by a,NaN,news,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20636,23861,Families also have solar lamps which can be re...,NaN,news,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22481,25874,Read the [full blog post](http://www.odi.org.u...,NaN,news,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23537,27071,Actualmente e independientemente de la ayuda d...,NaN,news,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- Observation : For `related=2` the original message is either NaN or from other Languages apart from English. Dopping this column for simplication

## 5. Dropping row which have related=2

In [20]:
df_all = df_all[df_all.related !=2]

In [21]:
df_all.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26182.000000,26182.000000,26182.000000,26182.000000,26182.000000,26182.000000,26182.000000,26182.000000,26182.000000,26182.000000,...,26182.000000,26182.000000,26182.000000,26182.000000,26182.000000,26182.000000,26182.000000,26182.000000,26182.000000,26182.000000
mean,15247.165725,0.765488,0.172370,0.004621,0.418379,0.080170,0.050378,0.027691,0.017989,0.032962,...,0.011802,0.044114,0.280460,0.083149,0.094110,0.010771,0.093996,0.020357,0.052670,0.195287
std,8838.361224,0.423702,0.377709,0.067826,0.493302,0.271561,0.218728,0.164088,0.132915,0.178539,...,0.107996,0.205353,0.449232,0.276112,0.291988,0.103224,0.291829,0.141223,0.223378,0.396429
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7436.250000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15752.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22964.750000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
df_all.nunique()

id                        25993
message                   25990
original                   9507
genre                         3
related                       2
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
child_alone                   1
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_in

- Observation : All Category columns should have number of unique values as 2 because we have values 0 or 1. But we can see `child alone` has only one unique value

# 6. Dropping child_alone column 

In [23]:
df_all['child_alone']

0        0
1        0
2        0
3        0
4        0
        ..
26381    0
26382    0
26383    0
26384    0
26385    0
Name: child_alone, Length: 26182, dtype: int64

`child_alone` category have all values a 0. so this can be dropped

In [24]:
df_all.drop(['child_alone'],axis=1,inplace=True)

In [25]:
df_all.nunique()

id                        25993
message                   25990
original                   9507
genre                         3
related                       2
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_infrastructure          2
weather_

## 7. Dropping duplicates from the dataset

In [26]:
df_all.drop_duplicates(subset=['message'], inplace=True)

In [27]:
# check number of duplicates
df_all[df_all.duplicated(subset='message')].count()

id                        0
message                   0
original                  0
genre                     0
related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake          

# Load this Clean dataset into sqlite Database

In [28]:
engine = create_engine('sqlite:///DisasterResponse.db')
df_all.to_sql('DS_messages', engine, index=False, if_exists='replace')

With the help of this notebook we can complete ETL Pipeline in `process_data.py` script